In [1]:
import json
import os
import cv2
import copy
from tqdm import tqdm

image_dir = './datasets/kaist-rgbt/train/images/visible'
label_dir = './datasets/kaist-rgbt/train/labels'

In [2]:
# Annotation format based on example file
anno_format_init = {
    "info": {
        "dataset": "KAIST Multispectral Pedestrian Benchmark",
        "url": "https://soonminhwang.github.io/rgbt-ped-detection/",
        "related_project_url": "http://multispectral.kaist.ac.kr",
        "publish": "CVPR 2015"
    },
    "info_improved": {
        "sanitized_annotation": {
            "publish": "BMVC 2018",
            "url": "https://li-chengyang.github.io/home/MSDS-RCNN/",
            "target": "files in train-all-02.txt (set00-set05)"
        },
        "improved_annotation": {
            "url": "https://github.com/denny1108/multispectral-pedestrian-py-faster-rcnn",
            "publish": "BMVC 2016",
            "target": "files in test-all-20.txt (set06-set11)"
        }
    },
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 0,
            "name": "person"
        },
        {
            "id": 1,
            "name": "cyclist"
        },
        {
            "id": 2,
            "name": "people"
        },
        {
            "id": 3,
            "name": "person?"
        }
    ]
}

In [3]:

kfold = 5

# Circle files in img directory
for i in range(1, kfold+1):
    anno_format = copy.deepcopy(anno_format_init)
    image_id = 0
    annotation_id = 0
    output_path = f"./utils/eval/KAIST_annotation_{i}.json"
    if os.path.exists(output_path):
        print(f"Annotation file for fold {i} already exists, remove and regenerating...")
        os.remove(output_path)

    list_path = f"./datasets/kaist-rgbt/val_all_04_{i}.txt"
    with open(list_path, 'r') as img_name:
        img_list = img_name.readlines()
        for list in tqdm(img_list, desc=f"Generating Annotations for fold {i}..."):
            file_name = os.path.basename(list).strip()
            image_path = os.path.join(image_dir, file_name)
            annotation_path = os.path.join(label_dir, file_name.replace('.jpg', '.txt'))
            
            #Read Image file, image should be read all
            img = cv2.imread(image_path)
            height, width, _ = img.shape
            image_info = {
                    "id": image_id,
                    "im_name": file_name,
                    "width": width,
                    "height": height      
                }
            anno_format['images'].append(image_info)

            # Read annotation file
            with open(annotation_path, 'r') as f:
                lines = f.readlines()
                
                #Skip annotation if no annotations for image
                if not lines:
                    #Should add image_id
                    image_id += 1
                    continue

                for line in lines:
                    cls, x_lefttop, y_lefttop, w, h, occlevel = map(float, line.strip().split())
                    bbox = [x_lefttop*width, y_lefttop*height, w*width, h*height]
                    annotation_info = {
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": cls,
                        "bbox": bbox,
                        "height": bbox[-1],
                        "occlusion": occlevel, 
                        "ignore": occlevel
                    }
                    anno_format['annotations'].append(annotation_info)
                    annotation_id += 1

                image_id += 1

    with open(output_path, 'w') as f:
        json.dump(anno_format, f, indent=2)

Annotation file for fold 1 already exists, remove and regenerating...


Generating Annotations for fold 1...: 100%|██████████| 2508/2508 [00:14<00:00, 170.28it/s]


Annotation file for fold 2 already exists, remove and regenerating...


Generating Annotations for fold 2...: 100%|██████████| 2508/2508 [00:13<00:00, 181.96it/s]


Annotation file for fold 3 already exists, remove and regenerating...


Generating Annotations for fold 3...: 100%|██████████| 2508/2508 [00:13<00:00, 181.97it/s]


Annotation file for fold 4 already exists, remove and regenerating...


Generating Annotations for fold 4...: 100%|██████████| 2507/2507 [00:13<00:00, 183.22it/s]


Annotation file for fold 5 already exists, remove and regenerating...


Generating Annotations for fold 5...: 100%|██████████| 2507/2507 [00:18<00:00, 139.13it/s]
